In [1]:
from requests import post
from app.analysis import Quote
query = 'query UnknownFile($limit_0:[FundLimitOptionInput!]!) {viewer {...Fe}} fragment Fe on Viewer {_screenFunds4s9d1a:screenFunds(filterCategory:["$$"],limit:$limit_0,first:100) {edges {node {symbol}}}}'
variables = {'limit_0':[{'type':'FUND_TOTAL_ASSETS','min':1000000000,'inclusive':True},{'type':'FUND_NET_EXPENSE_RATIO','max':1,'inclusive':True}]}
r = post('https://lens.m1finance.com/graphql', json={'query': query.replace('$$', 'Alternatives'), 'variables': variables}).json()
symbols = [n['node']['symbol'] for n in r['data']['viewer']['_screenFunds4s9d1a']['edges']]

In [2]:
quote = Quote(symbols, 120)
f'from {quote.start.date()} to {quote.end.date()} - {len(quote.data.columns)}'

'from 2019-02-25 to 2019-06-25 - 12'

In [8]:
stat = quote.statistics(21)
stat = stat[(stat['drawdown'] < 5) & (stat['21-shrp'] > 0.4)]
stat

,21-len,21-mean,21-std,21-shrp,drawdown
Symbols,,,,,
ICF,64,1.951614,2.130513,0.837802,4.165182
XLRE,64,2.098014,2.409941,0.801409,4.250690
VNQ,64,1.738370,2.057564,0.763866,3.975014
IYR,64,1.700149,2.054348,0.746457,3.822151
USRT,64,1.516771,1.867974,0.722764,4.162691
REET,64,1.149146,1.674407,0.586762,3.265309
SCHH,64,1.232048,1.832933,0.581244,4.332364
RWR,64,1.210802,1.814588,0.575412,4.320491


In [9]:
quote.setup_mask(stat.index)
quote.least_correlated_portfolio(21, 3, ['VNQ'])

['VNQ'] 0.8311366291897575 0.6 0.7950028117415554 0.7638661825517981
['VNQ', 'ICF', 'XLRE'] 0.7980233732926465 0.9729898090726357 0.8260590307954873 0.8010254665754765
['VNQ', 'ICF', 'IYR'] 0.7942024844533857 0.9794208540391457 0.7974898095689047 0.7827081791546647


['VNQ', 'ICF', 'IYR']

In [10]:
quote.setup_mask(['VNQ', 'ICF', 'IYR'])
quote.find_optimal_ratio(21, 2)

/Users/chaoyi/.local/share/virtualenvs/portfolio_analysis-bdMx2S3t/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


(Symbols
 VNQ   -0.072858
 ICF   -0.153780
 IYR    1.226638
 dtype: float64, 1.6587, 2.0529)